# Flickbike

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import sys
sys.path.append("../")
import KNMIRequest as KR


import warnings
warnings.filterwarnings('ignore')

## Importing the data
Allereerst wordt de data ingeladen.

In [ ]:
df = pd.read_csv('../input/bikes.csv', index_col=0, parse_dates=True)
df.head()

In [ ]:
df_KNMI = KR.get_KNMI_HH(start=2019010100, end=2019120923)
df_KNMI.head()

In [ ]:
df_KNMI["Hour"].value_counts()

In [ ]:
KR.data_dictionary

## Analysing the data

In [ ]:
df.dtypes

Omzetten naar correcte datatypes

In [ ]:
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
df['date'] = pd.to_datetime(df['date'])

# extracting extra features
df['weekday'] = df['date'].dt.day_name()
df['hour'] = df['date'].dt.hour

df.dtypes

In [ ]:
df.isna().sum()

## Verdiepingsopdracht

### Vraag 1
**Produceer een plot van verhuur-per-dag voor deze dataset.**  
  
Om dit voor elkaar te krijgen worden eerst alle dagen bij elkaar gegroupeerd zodat de hoeveelheid verhuren per dag opgeteld zullen worden.  
Hierna wordt deze dataset geplot.

In [ ]:
df_dates_count = df.groupby(df['date'].dt.date)['tripid'].count()

x = df_dates_count.index
y = df_dates_count.values

# sns.set_style("darkgrid")
plt.plot(x, y)

plt.title("")
plt.xlabel("")
plt.ylabel("")

plt.figure(figsize=(18,18))
plt.show()

In [ ]:
df_dates_count.index = pd.to_datetime(df_dates_count.index)
df_three_months = df_dates_count.loc["2019-09-01":"2019-12-31"]

# sns.set_style("darkgrid")
plt.plot(df_three_months.index, df_three_months.values)

plt.title("")
plt.xlabel("")
plt.ylabel("")

plt.xticks(rotation=45)
plt.figure(figsize=(18,18))
plt.show()

### Vraag 2
**Wat is de periodiciteit van deze dataset?**  
  
Om de vraag wat betreft periodiciteit te beantwoorden zullen we moeten gaan kijken of de data zich volgens een bepaald patroon gedraagt. Om deze vraag te beantwoorden bouwen we eerst een autocorrelatie plot.


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(df_dates_count.values, lags=30)
plt.show()

# Opleveropdracht
### 1. Hoe voorspel je het aantal verhuurde fietsen op de dagen tussen 10 december 2019 en 31 januari 2020?

Voordat een model gebouwd wordt, moet er data verzameld worden. Op dit moment is de data over alle verhuurde fietsen in de regio Schiphol-Oost in de tijdsperiode vanaf 1 januari 2019 tot 9 december 2019 beschikbaar gesteld. 

In de data staan gegevens van id’s van gebruikers, fietsen en trips. Verder bevat de data ook informatie over datum en tijd van gebruik, en de coördinaten van de startpunten en eindpunten van iedere trip.


In [ ]:
df.head()

### 2. Welke invloeden verwacht je van, en op welke manier kun je je model gevoelig maken voor factoren?
Om het model gevoeliger te maken voor factoren moet eerst besloten worden welke factoren invloed kunnen hebben over de uiteindelijke uitkomst van de voorspelling. Een aantal mogelijke factoren zijn al gegeven:
1. Het weer.
2. De dag van de week.
3. Bijzondere dagen in het jaar, zoals feestdagen, schoolvakanties, etc.
4. Eventueel andere factoren.

Per bovenstaande factor zullen we nu de mogelijke invloed beschrijven:
#### Het weer

* Regen	         
    * Iemand die gewoonlijk loopt fietst nu
    * Iemand die gewoonlijk fietst neemt shuttle/taxi
* Zonneschijn 	 
    * Iemand die gewoonlijk fietst, kan gaan lopen
    * Meer mensen fietsen
* Onweer
    * Mensen lopen eerder, zodat ze niet te groot zijn (minder kans op inslag)
* Mistigheid
    * Mensen fietsen wellicht minder bij weinig zicht 

##### gevoeligheid model
Het model valt gevoellig te maken voor weer door een aantal weersverschijnselen direct als variabelen mee te nemen. De volgende data komt direct van het KNMI, en gaat over de gegeven tijdspanne.

In [ ]:
df_KNMI.head()

In [ ]:
df_KNMI.describe()

#### De dag van de week
* Woensdag/vrijdag 	
  * → Vaak gebruikt als thuiswerkdagen 		
* Zaterdag/zondag 	
   * Weekend, geen werk, zwaar verminderd gebruik. 

In [ ]:
df_weekday = df_dates_count.copy(deep=True)
df_weekday.index = df_weekday.index.day_name()

df_weekday.groupby(df_weekday.index).mean()

In [ ]:
sns.boxplot(x=df_weekday.index, y=df_weekday.values)
plt.show()

##### Gevoeligheid model


#### Bijzondere dagen in het jaar
* Feestdagen 		
  * ‘verplicht’ vrij, geen werk
* Schoolvakanties 
    * Minder fietsen benodigd, x percentage werkende mensen met vakantie.

Eventuele andere factoren
Bij eventuele andere factoren kan gedacht worden aan factoren die, tot nu toe, nog niet benoemd zijn maar misschien wel een invloed kunnen hebben op de uitslag van de voorspelling. Bij dit soort factoren kan gedacht worden aan:
#### Uur van de dag
* Woensdag	
  * Kinderen eerder uit, eerder weg
* Vrijdag 		
  * Eerder vrij of vrijmibo, eerder/later weg
* Spitstijd	
  * Meer fietsen benodigd in de spits dan erbuiten.
* Lunchpauze 
  * Mensen pakken de fiets elders hun lunchpauze te houden.

In [ ]:
df_dates_count.sort_values(ascending=False).head(2)

Dit geeft aan dat er op de 5e van november in totaal 499 fietsen verhuurt zijn. De volgende stap is om te kijken rond welke uren de vraag het hoogst is.

In [ ]:
df_1day_hours = df[(df['date'] > '2019-11-05') & (df['date'] < '2019-11-6')]
df_plot = df_1day_hours.groupby(df_1day_hours['hour'])['tripid'].count()

# plotting the bar chart
sns.set_style("darkgrid")

y_pos = np.arange(len(df_plot.index))
plt.bar(y_pos, df_plot.values, align='center', alpha=0.5)

plt.xticks(y_pos, df_plot.index)
plt.ylabel('')
plt.title('')

plt.show()